In [1]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Converting a Non-Deterministic <span style="font-variant:small-caps;">Fsm</span> into a Deterministic <span style="font-variant:small-caps;">Fsm</span>

In this notebook we show how a non-deterministic <span style="font-variant:small-caps;">Fsm</span>
$$ F = \langle Q, \Sigma, \delta, q_0, A \rangle $$
can be transformed into a deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ such that both <span style="font-variant:small-caps;">Fsm</span>s accept the
same language, that is we have
$$ L(F) = L\bigl(\texttt{det}(F)\bigr). $$
The idea behind this transformation is that the <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ has to 
compute the set of all states that the <span style="font-variant:small-caps;">Fsm</span> $F$ could be in. 
Hence the states of the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ are 
**sets** of states of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $F$.  A set of these states contains all those states that the non-deterministic <span style="font-variant:small-caps;">Fsm</span> 
$F$ could have reached.  Furthermore, a set $M$ of states of the <span style="font-variant:small-caps;">Fsm</span> $F$ is an accepting state of the <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ if the set $M$ contains an accepting state of the <span style="font-variant:small-caps;">Fsm</span> $F$.

## Declaring the Necessary Types

To implement the mathematical concepts of Finite Automata efficiently in TypeScript, we utilize the `RecursiveSet` library. This library allows us to treat sets as values: equality is determined by content, not reference. This is essential for the Subset Construction algorithm, where states of the DFA are sets of NFA states.

In [2]:
import { RecursiveSet, RecursiveMap, Tuple } from "./recursive-set";

### Basic Types

We define the fundamental building blocks of our automata:

1.  **Atomic States (`State`):**
    Identifiers for the states of the NFA (e.g., strings or numbers).

2.  **Input Symbols (`Char`):**
    Single characters from the alphabet $\Sigma$.

3.  **Composite States (`DFAState`):**
    For the **Subset Construction** algorithm, a state in the resulting DFA is defined as a **set** of NFA states. We define this explicit type alias to distinguish clearly between atomic states ($q \in Q_{NFA}$) and composite states ($S \subseteq Q_{NFA}$).

In [3]:
type State = string | number;

type Char = string;

type DFAState = RecursiveSet<State>;

### Transition Relations

We distinguish between the transition functions for NFAs and DFAs based on their codomain (return type).

**1. NFA Transition ($\delta_{NFA}$):**
The transition function maps a state and a character to a **set** of possible next states. This reflects the non-deterministic nature of the automaton.
$$\delta: Q \times \Sigma \to \mathcal{P}(Q)$$

In [4]:
type TransRel = RecursiveMap<Tuple<[State,Char]>, RecursiveSet<State>>;

**2. DFA Transition ($\delta_{DFA}$):**
The deterministic transition function maps a state and a character to exactly **one** next state.
$$\delta: Q_{DFA} \times \Sigma \to Q_{DFA}$$

In [5]:
type TransRelDet = RecursiveMap<Tuple<[DFAState,Char]>, DFAState>;

### Automata Definitions

Finally, we define the object structures for the Non-Deterministic (NFA) and Deterministic (DFA) Finite Automata.

* **NFA:** A 5-tuple $A = (Q, \Sigma, \delta, q_0, F)$ where $\delta$ allows multiple targets.

In [6]:
type NFA = {
    Q: RecursiveSet<State>;
    Σ: RecursiveSet<Char>;
    δ: TransRel;
    q0: State;
    A: RecursiveSet<State>;
};

* **DFA:** A 5-tuple $A' = (Q', \Sigma, \delta', q'_0, F')$ where the states in $Q'$ are of type `DFAState` (sets).

In [7]:
type DFA = {
    Q: RecursiveSet<DFAState>;
    Σ: RecursiveSet<Char>;
    δ: TransRelDet;
    q0: DFAState;
    A: RecursiveSet<DFAState>;
};

<hr style="height:5px;background-color:blue">

## Subset Construction Algorithm

In order to present the construction of $\texttt{det}(F)$ we first have to define three auxiliary functions.

We start with the function `bigUnion`. Given a set `M` that contains sets of elements (i.e., a set of sets), the expression `bigUnion(M)` returns the union of all sets in `M`. Formally:
$$\texttt{bigUnion}(M) = \bigcup M = \bigl\{ x \bigm| \exists A \in M: x \in A \bigr\}.$$

In [8]:
const bigUnion = (M: RecursiveSet<DFAState>): DFAState => {
    const res = new RecursiveSet<State>();
    for (const A of M) for (const x of A) res.add(x);
    return res;
};

We can test `bigUnion()` below:

In [16]:
const input = new RecursiveSet<DFAState>(
    new RecursiveSet(1, 2, 3),
    new RecursiveSet(2, 3, 4),
    new RecursiveSet(3, 4, 5),
);
bigUnion(input);

{1, 2, 3, 4, 5}


The function `epsClosure` takes two arguments:
- `s` is a state, 
- `delta` is the transition function of the NFA $F$.

The function computes the set of all those states that can be reached from the state `s` via $\varepsilon$-transitions.
Formally, the set $\texttt{epsClosure}(s)$ is defined inductively:
- $s \in \texttt{epsClosure}(s)$.
- $p \in \texttt{epsClosure}(s) \wedge r \in \delta(p, \varepsilon) \;\rightarrow\; r \in \texttt{epsClosure}(s)$.
 
If the state $p$ is an element of the $\varepsilon$-closure of the state $s$ and there is an $\varepsilon$-transition from $p$ to some state $r$, then $r$ is also an element of the $\varepsilon$-closure of $s$.
 
The implementation of `epsClosure` uses a **fixed-point algorithm**. We start with $\{s\}$ and iteratively add reachable states until the set stabilizes.

In [10]:
const epsClosure = (s: State, δ: TransRel): DFAState => {
    let Res = new RecursiveSet(s);
    while (true) {
        const Reachable = new RecursiveSet<DFAState>();
        for (const q of Res) {
            const targets = δ.get(new Tuple(q, "ε"));
            if (targets) Reachable.add(targets);
        }
        
        const New = bigUnion(Reachable);
        if (New.isSubset(Res)) return Res;
        Res = Res.union(New);
    }
};

In order to transform a non-deterministic <span style="font-variant:small-caps;">Fsm</span> $F$ into a deterministic 
<span style="font-variant:small-caps;">Fsm</span>
$\texttt{det}(F)$ we have to extend the function $\delta:Q \times \Sigma \rightarrow 2^Q$ into the function
$$\widehat{\delta}: Q \times \Sigma \rightarrow 2^Q. $$
The idea is that given a state $q$ and a character $c$,  the value of $\widehat{\delta}(q,c)$ is the set of all states that the
<span style="font-variant:small-caps;">Fsm</span> $F$ could reach when it reads the character $c$ in state $q$ and then performs an arbitrary number of $\varepsilon$-transitions.  Formally, the definition of $\widehat{\delta}$ is as follows:
$$ \widehat{\delta}(q_1, c) := \bigcup \bigl\{ \texttt{epsClosure}(q_2) \bigm| q_2 \in \delta(q_1, c) \bigr \}. $$
This formula is to be read as follows:
- For every state $q_2 \in Q$ that can be reached from the state $q_1$ by reading the character $c$ we
  compute $\texttt{epsClosure}(q_2)$.
- Then we take the union of all these sets $\texttt{epsClosure}(q_2)$.

The function $\widehat{\delta}$ is implemented as the function `deltaHat`, which takes three arguments:
- `s` is a state,
- `c` is a character,
- `𝛿` is the transition function of a non-deterministic 
  <span style="font-variant:small-caps;">Fsm</span>.

This function computes the set of all those states that can be reached 
from `s` when we first have a transition from state `s` to some state `p` 
on reading the character `c` followed by any number of $\varepsilon$-transitions
starting in `p`.

In [11]:
const deltaHat = (s: State, c: Char, δ: TransRel): DFAState => {
    const targets = δ.get(new Tuple(s, c));
    if (!targets) return new RecursiveSet();
    
    const closures = new RecursiveSet<DFAState>();
    for (const q of targets) closures.add(epsClosure(q, δ));
    return bigUnion(closures);
};

The function $\widehat{\delta}$ maps a state into a set of states. Since the FSM $\texttt{det}(F)$ uses sets of states of the FSM $F$ as its states, we need a function that maps sets of states into sets of states. Hence we generalize the function $\widehat{\delta}$ to the function
$$\Delta: 2^Q \times \Sigma \rightarrow 2^Q$$
such that for a set $M$ of states and a character $c$ the expression $\Delta(M, c)$ computes the set of all those states that the FSM $F$ could be in if it is in a state from the set $M$, then reads the character $c$, and finally makes some $\varepsilon$-transitions.

The formal definition is as follows: 
$$\Delta(M,c) := \bigcup \bigl\{ \widehat{\delta}(q,c) \bigm| q \in M \bigr\}.$$

This formula is easy to understand: For every state $q \in M$ we compute the set of states that the FSM could be in after reading the character $c$ and doing some $\varepsilon$-transitions. Then we take the union of these sets.

In [12]:
const capitalDelta = (M: DFAState, c: Char, δ: TransRel): DFAState => {
    const sets = new RecursiveSet<DFAState>();
    for (const q of M) sets.add(deltaHat(q, c, δ));
    return bigUnion(sets);
};

The function `allStates` takes three arguments:
- $Q$ is $\texttt{epsClosure}(q_0)$, where $q_0$ is the start state of the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$,
- $\delta$ is the transition function of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $F$, and
- $\Sigma$ is the alphabet of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $F$.

The function `allStates` computes the set of all states of the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ that can be reached from the start state.

In [13]:
const allStates = (Q0: DFAState, δ: TransRel, Σ: RecursiveSet<Char>): RecursiveSet<DFAState> => {
    let Res = new RecursiveSet(Q0);
    while (true) {
        const New = new RecursiveSet<DFAState>();
        for (const M of Res) {
            for (const c of Σ) {
                New.add(capitalDelta(M, c, δ));
            }
        }
        if (New.isSubset(Res)) return Res;
        Res = Res.union(New);
    }
};

Now we are ready to formally define how the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$
is computed from the non-deterministic <span style="font-variant:small-caps;">Fsm</span>
$F = \bigl\langle Q, \Sigma, \delta, q_0, A \bigr\rangle$.
We define: 
$$ \texttt{det}(F) := \bigl\langle \texttt{allStates}(\texttt{epsClosure}(q_0)), \Sigma, \Delta, \texttt{epsClosure}(q_0), \widehat{A} \bigr\rangle $$
where the components of this tuple are given as follows:
- The set of states of $\texttt{det}(F)$ is the set of all states that can be reached from the set $\texttt{epsClosure}(q_0)$.
- The input alphabet $\Sigma$ does not change when going from $F$ to $\texttt{det}(F)$.
  After all, the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ has to recognize the same language as the non-deterministic
  <span style="font-variant:small-caps;">Fsm</span> $F$.
- The function $\Delta$, that has been defined previously, specified how the set of states change when a
  character is read.
- The start state $\texttt{epsClosure}(q_0)$ of the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ is the set of all states
  that can be reached from the start state $q_0$ of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $F$
  via $\varepsilon$-transitions.
- The set of accepting states $\widehat{A}$ is the set of those subsets of $Q$ that contain an accepting
  state of the <span style="font-variant:small-caps;">Fsm</span> $F$:
  $$\widehat{A} := \bigl\{ M \in 2^Q \mid M \cap A \not= \{\} \bigl\}. $$
  
### Implementation Note: Lazy Construction
Although the powerset $2^Q$ can be exponentially large ($2^n$ states), we do not construct the entire powerset. Instead, our `allStates` function ensures we only construct the **reachable** part of the DFA. This makes the conversion efficient for many practical NFAs.

In [14]:
const nfa2dfa = (nfa: NFA): DFA => {
    const { Σ, δ, q0, A } = nfa;
    const start = epsClosure(q0, δ);
    const Q_DFA = allStates(start, δ, Σ);
    
    const δ_DFA = new RecursiveMap<Tuple<[DFAState, Char]>, DFAState>();
    
    for (const M of Q_DFA) {
        for (const c of Σ) {
            δ_DFA.set(new Tuple(M, c), capitalDelta(M, c, δ));
        }
    }

    const A_DFA = new RecursiveSet<DFAState>();
    for (const M of Q_DFA) {
        if (!M.intersection(A).isEmpty()) A_DFA.add(M);
    }

    return { Q: Q_DFA, Σ, δ: δ_DFA, q0: start, A: A_DFA };
};

To test this function, use the notebook `02-Test-NFA-2-DFA.ipynb`.

In [17]:
import { RecursiveSet, RecursiveMap, Tuple } from "./recursive-set";
import { nfa2dfa, NFA, TransRel } from "./01-NFA-2-DFA";

// 1. NFA SETUP
const States = new RecursiveSet(...Array.from({ length: 8 }, (_, i) => `q${i}`));
const Sigma = new RecursiveSet("a", "b");
const delta: TransRel = new RecursiveMap();

// Transitions definieren
console.log("🛠️ Building NFA Delta...");
delta.set(new Tuple("q0", "ε"), new RecursiveSet("q1", "q2"));
delta.set(new Tuple("q1", "b"), new RecursiveSet("q3"));
delta.set(new Tuple("q2", "a"), new RecursiveSet("q4"));
delta.set(new Tuple("q3", "a"), new RecursiveSet("q5"));
delta.set(new Tuple("q4", "b"), new RecursiveSet("q6"));
delta.set(new Tuple("q5", "ε"), new RecursiveSet("q7"));
delta.set(new Tuple("q6", "ε"), new RecursiveSet("q7"));
delta.set(new Tuple("q7", "ε"), new RecursiveSet("q0"));

// Check: Wurde Delta korrekt befüllt?
console.log(`NFA Delta Size: ${delta.size}`);
console.log(delta.toString());

const nfa: NFA = {
    Q: States,
    Σ: Sigma,
    δ: delta,
    q0: "q0",
    A: new RecursiveSet("q7"),
};

// 2. CONVERSION
console.log("\n⚙️ Running nfa2dfa()...");
const dfa = nfa2dfa(nfa);

// 3. MANUAL INSPECTION
console.log("\n--- DFA RESULT ANALYSIS ---");

// A. Start State Check
// Erwartung: epsClosure(q0) -> {q0, q1, q2}
console.log("DFA Start State (q0):");
console.log(dfa.q0.toString()); 

// B. States Check
console.log(`\nDFA States (|Q| = ${dfa.Q.size}):`);
// Wir iterieren manuell, um sicherzugehen
for (const s of dfa.Q) {
    console.log(`  State: ${s.toString()}`);
    if (dfa.A.has(s)) {
        console.log(`    -> ACCEPTING (contains q7)`);
    }
}

// C. Transitions Check
console.log(`\nDFA Transitions (|delta| = ${dfa.δ.size}):`);
// Iteration über die RecursiveMap
for (const [keyTuple, targetState] of dfa.δ) {
    // Key ist Tuple<[Set<State>, Char]>
    const fromState = keyTuple.get(0);
    const char = keyTuple.get(1);
    
    console.log(`  δ( ${fromState.toString()}, '${char}' )  -->  ${targetState.toString()}`);
}

🛠️ Building NFA Delta...
NFA Delta Size: 8
RecursiveMap(8) {
  (q0, ε) => {q1, q2},
  (q1, b) => {q3},
  (q2, a) => {q4},
  (q3, a) => {q5},
  (q4, b) => {q6},
  (q5, ε) => {q7},
  (q6, ε) => {q7},
  (q7, ε) => {q0}
}

⚙️ Running nfa2dfa()...

--- DFA RESULT ANALYSIS ---
DFA Start State (q0):
{q0, q1, q2}

DFA States (|Q| = 6):
  State: {q0, q1, q2}
  State: {q4}
  State: {q3}
  State: {}
  State: {q0, q1, q2, q6, q7}
    -> ACCEPTING (contains q7)
  State: {q0, q1, q2, q5, q7}
    -> ACCEPTING (contains q7)

DFA Transitions (|delta| = 12):
  δ( {q0, q1, q2}, 'a' )  -->  {q4}
  δ( {q0, q1, q2}, 'b' )  -->  {q3}
  δ( {q4}, 'a' )  -->  {}
  δ( {q4}, 'b' )  -->  {q0, q1, q2, q6, q7}
  δ( {q3}, 'a' )  -->  {q0, q1, q2, q5, q7}
  δ( {q3}, 'b' )  -->  {}
  δ( {}, 'a' )  -->  {}
  δ( {}, 'b' )  -->  {}
  δ( {q0, q1, q2, q6, q7}, 'a' )  -->  {q4}
  δ( {q0, q1, q2, q6, q7}, 'b' )  -->  {q3}
  δ( {q0, q1, q2, q5, q7}, 'a' )  -->  {q4}
  δ( {q0, q1, q2, q5, q7}, 'b' )  -->  {q3}


In [18]:
nfa;

{
  Q: {q0, q1, q2, q3, q4, q5, q6, q7},
  'Σ': {a, b},
  'δ': RecursiveMap(8) {
    (q0, ε) => {q1, q2},
    (q1, b) => {q3},
    (q2, a) => {q4},
    (q3, a) => {q5},
    (q4, b) => {q6},
    (q5, ε) => {q7},
    (q6, ε) => {q7},
    (q7, ε) => {q0}
  },
  q0: 'q0',
  A: {q7}
}


In [19]:
dfa;

{
  Q: {{q4}, {q0, q1, q2, q5, q7}, {q0, q1, q2, q6, q7}, {q0, q1, q2}, {q3}, {}},
  'Σ': {a, b},
  'δ': RecursiveMap(12) {
    ({}, a) => {},
    ({}, b) => {},
    ({q0, q1, q2, q5, q7}, a) => {q4},
    ({q0, q1, q2, q5, q7}, b) => {q3},
    ({q0, q1, q2, q6, q7}, a) => {q4},
    ({q0, q1, q2, q6, q7}, b) => {q3},
    ({q0, q1, q2}, a) => {q4},
    ({q0, q1, q2}, b) => {q3},
    ({q3}, a) => {q0, q1, q2, q5, q7},
    ({q3}, b) => {},
    ({q4}, a) => {},
    ({q4}, b) => {q0, q1, q2, q6, q7}
  },
  q0: {q0, q1, q2},
  A: {{q0, q1, q2, q5, q7}, {q0, q1, q2, q6, q7}}
}
